In [ ]:
import logging
from kiteconnect import KiteConnect
logging.basicConfig(level=logging.ERROR)
import threading
kite = KiteConnect(api_key="w19o0chuo929jxkp")
import joblib
kite = joblib.load('kitefile.p')
kite.login_url()

In [ ]:
#####################################################
mytoken = '1BWgO1UgTFeEiE1gS0SttoVTXT4dgcAg'        
#####################################################
data = kite.generate_session(mytoken,api_secret="gsw8ps17ex7lf3cuji4prfnwb4vlyr4y")
kite.set_access_token(data["access_token"])
import joblib
joblib.dump(kite,'kitefile.p')

In [ ]:
import act

In [ ]:
kite.orders()[-1]

In [ ]:
kite.positions()['net']

In [ ]:
kite.holdings()

In [1]:
#####################
pivot_price = 73.3425
last_high = 73.7950
last_low = 73.1875
fullquantity = 70
powermulti = 1.5
#####################

total_parts = 24
pivot_range = last_high - last_low
scounter = int(total_parts/2)

fibonmulti1=0.1180
fibonmulti2=0.2360
fibonmulti3=0.3820
fibonmulti4=0.6180
fibonmulti5=1.0000  # 100% Range
fibonmulti6=1.1180
fibonmulti7=1.2360
fibonmulti8=1.3820
fibonmulti9=1.6180
fibonmulti10=2.0000 # Max Quantity @ 200% Range
fibonmulti11=2.1180 # Stop Trading here
fibonmulti12=2.3820 # STOPLOSS

print()
print('pivot price : ' +str(pivot_price))

print()
base=0.0025

for i in range(1, int(total_parts/2)+1):
    globals()['p'+str(i)+'b'] = round(pivot_price - pivot_range*(globals()['fibonmulti'+str(i)]),4)
    globals()['p'+str(i)+'b'] = round(base * round(globals()['p'+str(i)+'b'] /base),4)
    print('p'+str(i)+'b = '+str(globals()['p'+str(i)+'b']))

print()
for i in range(1, int(total_parts/2)+1):
    globals()['p'+str(i)+'s'] = round(pivot_price + pivot_range*(globals()['fibonmulti'+str(i)]),4)
    globals()['p'+str(i)+'s'] = round(base * round(globals()['p'+str(i)+'s'] /base),4)
    print('p'+str(i)+'s = '+str(globals()['p'+str(i)+'s']))

print()

halfqunat = fullquantity/2
tempcounter = 1

for j in range(2):
    for i in range(4,-1,-1):
        globals()['p'+str(tempcounter)+'q'] = int((halfqunat*j) + int(halfqunat/pow(powermulti,i)))
        print('p'+str(tempcounter)+'q : ' + str(globals()['p'+str(tempcounter)+'q']))
        tempcounter = tempcounter+1

p11q = p10q
p12q = 0
print('p11q : '+ str(p11q))
print('p12q : '+ str(p12q))


pivot price : 73.3425

p1b = 73.27
p2b = 73.2
p3b = 73.11
p4b = 72.9675
p5b = 72.735
p6b = 72.6625
p7b = 72.5925
p8b = 72.5025
p9b = 72.36
p10b = 72.1275
p11b = 72.055
p12b = 71.895

p1s = 73.415
p2s = 73.485
p3s = 73.575
p4s = 73.7175
p5s = 73.95
p6s = 74.0225
p7s = 74.0925
p8s = 74.1825
p9s = 74.325
p10s = 74.5575
p11s = 74.63
p12s = 74.79

p1q : 6
p2q : 10
p3q : 15
p4q : 23
p5q : 35
p6q : 41
p7q : 45
p8q : 50
p9q : 58
p10q : 70
p11q : 70
p12q : 0


In [ ]:
myquantity = 0
def getquant():
    global myquantity
    allpos = kite.positions()['net']
    for i in range(len(allpos)):
        if(allpos[i]['tradingsymbol']=='USDINR20OCTFUT'):
            myquantity = allpos[i]['quantity']
            print('My Quantity : ' + str(allpos[i]['quantity']))
        
getquant()

def placeneworder(quantdiff):
    global stopbuy
    global stopsell
    if(quantdiff>0 and stopbuy==False):
        try:
                order_id= kite.place_order(tradingsymbol="USDINR20OCTFUT",
                            exchange=kite.EXCHANGE_CDS,
                            transaction_type=kite.TRANSACTION_TYPE_BUY,
                            quantity=abs(quantdiff),
                            order_type=kite.ORDER_TYPE_MARKET,
                            variety = kite.VARIETY_REGULAR,
                            product=kite.PRODUCT_NRML)
                getquant()
                stopsell=False
        except Exception as e:
            stopbuy=True
            print(e)
            getquant()
    if(quantdiff<0 and stopsell==False):
        try:
                order_id= kite.place_order(tradingsymbol="USDINR20OCTFUT",
                            exchange=kite.EXCHANGE_CDS,
                            transaction_type=kite.TRANSACTION_TYPE_SELL,
                            quantity=abs(quantdiff),
                            order_type=kite.ORDER_TYPE_MARKET,
                            variety = kite.VARIETY_REGULAR,
                            product=kite.PRODUCT_NRML)
                getquant()
                stopbuy=False
        except Exception as e:
            stopsell=True
            print(e)
            getquant()


def ckqnt (orderprice,orderquant):
    global myquantity
    quantdiff = orderquant - myquantity
    placeneworder(quantdiff)
    return myquantity


def checkpivot(ltps):
    if ((ltps>=(pivot_price-0.0050)) and (ltps<=(pivot_price+0.0050))):
        ckqnt(pivot_price,0)
        print('in pivot range')
        
    for i in range(1,int(total_parts/2)+1):
        if(i==int(total_parts/2)):
            if (ltps <= (globals()['p'+str(i)+'b']-0.0025)):
                ckqnt(globals()['p'+str(i)+'b'],globals()['p'+str(i)+'q'])
                print('==========' + 'p'+str(i)+'b' +'===========')
                print(str(ltps)+ ' < ' + 'p'+str(i)+'b')
            if (ltps >= (globals()['p'+str(i)+'s']+0.0025)):
                ckqnt(globals()['p'+str(i)+'s'],-(globals()['p'+str(i)+'q']))
                print('==========' + 'p'+str(i)+'b' +'===========')
                print(str(ltps)+ ' < ' + 'p'+str(i)+'b')
        else:
            if (ltps < (globals()['p'+str(i)+'b'] -0.0025) and ltps > (globals()['p'+str(i+1)+'b']+0.0025)):
                ckqnt(globals()['p'+str(i)+'b'],globals()['p'+str(i)+'q'])
                print('==========' +'p'+str(i)+'b' + '==============' )
                print(str(globals()['p'+str(i+1)+'b'])+ ' < ' +str(ltps)+ ' < ' +str(globals()['p'+str(i)+'b']))
            if (ltps > (globals()['p'+str(i)+'s'] +0.0025) and ltps < (globals()['p'+str(i+1)+'s']-0.0025)):
                ckqnt(globals()['p'+str(i)+'s'],-(globals()['p'+str(i)+'q']))
                print('==========' +'p'+str(i)+'s' + '==============' )
                print(str(globals()['p'+str(i)+'s'])+ ' < ' +str(ltps)+ ' < ' +str(globals()['p'+str(i+1)+'s']))


In [ ]:
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#              START TRADING             !
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

import logging
from kiteconnect import KiteTicker
logging.basicConfig(level=logging.DEBUG)
kws = KiteTicker("w19o0chuo929jxkp", "8B3hH04gpZlVlCKbszhsNUTfhKW2o0E8")

def on_ticks(ws, ticks):
    checkpivot(ticks[0]['last_price'])
    #print(type(ticks[0]['last_price']))
    print('==========================')
    print( 'LTP : ' + str(ticks[0]['last_price']))
    print('==========================')
    print()

def on_connect(ws, response):
    ws.subscribe([690691])
    ws.set_mode(ws.MODE_FULL, [690691])

def on_close(ws, code, reason):
    ws.stop()

kws.on_ticks = on_ticks
kws.on_connect = on_connect
kws.on_close = on_close

kws.connect()